### Convert coefficients of parameterization from F77 to F90

Coefficients are originally declared in the offline code, which is in F77.  In order to use these in the global model, they need to be re-expressed in F90, before being inserted into the global model source code.

In [21]:
import os
import io
import collections
import re
import itertools

import numpy as np
import pandas as pd
import xarray as xr

In [192]:
dir_f77 = '/nuwa_cluster/home/jackyu/radiation/clirad/LW/src'
files_trans = {'h2o': 'h2o_tran3.F',
               'co2': 'co2_tran4.F',
               'o3': 'o3_tran3.F'}

band1_trans = dict(h2o=[1, 2, 8], co2=[1], o3=[1])
band2_trans = dict(h2o=[1, 2, 3], co2=[1, 2, 3], o3=[1, 2, 3])
num_pressures_trans = dict(h2o=26, co2=26, o3=26)
num_amounts_trans = dict(h2o=31, co2=30, o3=21)

In [221]:
gas = 'o3'

In [233]:
def load_gas_trans(gas='h2o', dir_f77='./'):
    '''
    Get gas transmission values from CLIRAD-LW offline model and
    store them in xarray.DataArray
    
    Parameters
    ----------
    gas: name of the gas: h2o, co2, or o3
    dir_f77: directory containing F77 files containing transmissions
    '''
    with open(os.path.join(dir_f77, files_trans[gas]), 
              encoding='utf-8', mode='r') as file:
        content = file.read()
        
    lines_data = (l for l in content.split('\n') if '&' in l)
    str_data = ''.join(lines_data)
    str_data = str_data.replace('&', '').replace('/', '')\
               .replace(',', '')
    
    df = pd.read_csv(io.StringIO(str_data), sep=r'\s+', header=None)
    data = np.squeeze(df).values
    
    num_band1s = len(band1_trans[gas])
    num_band2s = len(band2_trans[gas])
    num_pressures = num_pressures_trans[gas]
    num_amounts = num_amounts_trans[gas]
    
    data_ibs = data.reshape(num_band1s, int(data.shape[0] / num_band1s))
    
    pressure = range(1, num_pressures + 1)
    amount = range(1, num_amounts + 1)

    list_da_band1s = [xr.DataArray(data_ib.reshape(num_pressures, 
                                                   num_band2s, 
                                                   num_amounts),
                                   dims=['pressure', 'band2', 'amount'],
                                   coords=[pressure, 
                                           band2_trans[gas], 
                                           amount]) 
                      for data_ib in data_ibs]

    da = xr.concat(list_da_band1s, dim='band1')
    da.coords['band1'] = ('band1', band1_trans[gas])
    return da

In [247]:
da_h2o = load_gas_trans(gas='h2o', dir_f77=dir_f77)

In [236]:
da_co2 = load_gas_trans(gas='co2', dir_f77=dir_f77)

In [240]:
da_o3 = load_gas_trans(gas='o3', dir_f77=dir_f77)

In [418]:
def str_iw(da=None, gas='h2o', band1=1, band2=1, pressure=1):
    '''
    Returns string for gas transmittance for some gas, band1, band2
    and pressure in F90 syntax.
    
    Parameters
    ----------
    da: xarray.DataArray containing transmittance for gas.
    gas: name of the gas corresponding to da.
    band1: band 1. Can be 1, 2, or 8 for h2o.  
           Can only be 1 for co2 and o3.
    band2: band 2.  Can be 1, 2, or 3.
    pressure: pressure interval.
    '''
    writedata = list(da.sel(band1=band1, 
                            band2=band2,
                            pressure=pressure).values)

    fmt_num = ' {:> 12e}_r8'

    if gas == 'h2o':
        str_gas = 'h{band1}{band2}'.format(band1=band1, band2=band2)
    elif gas == 'co2':
        str_gas = 'c{band2}'.format(band2=band2)
    elif gas == 'o3':
        str_gas = 'o{band2}'.format(band2=band2)
    
    numbers_lines = []
    while True:
        numbers = []
        try:
            for _ in range(3):
                numbers.append(writedata.pop(0))
        except IndexError:
                if numbers:
                    numbers_lines.append(numbers)
                break
        else:
            numbers_lines.append(numbers)


    str_lines = []        
    for nums in numbers_lines[:-1]:
        str_line = ','.join(fmt_num.format(num) for num in nums)
        str_line += ', &'
        str_lines.append(str_line)

    str_line = ','.join(fmt_num.format(num) for num in numbers_lines[-1])
    str_line += '/)'
    str_lines.append(str_line)

    str_line = '{} ({:2d}, :) = (/ &'.format(str_gas, pressure)
    str_lines.insert(0, str_line)    
    
    return '\n'.join(str_lines)

In [422]:
print(str_iw(da_h2o, band1=8, band2=1, pressure=21))

h81 (21, :) = (/ &
  9.999864e-01_r8,  9.999729e-01_r8,  9.999461e-01_r8, &
  9.998930e-01_r8,  9.997890e-01_r8,  9.995871e-01_r8, &
  9.992027e-01_r8,  9.984920e-01_r8,  9.972310e-01_r8, &
  9.951140e-01_r8,  9.917710e-01_r8,  9.867800e-01_r8, &
  9.796300e-01_r8,  9.696200e-01_r8,  9.557300e-01_r8, &
  9.365800e-01_r8,  9.103700e-01_r8,  8.750000e-01_r8, &
  8.280000e-01_r8,  7.673000e-01_r8,  6.911000e-01_r8, &
  5.993000e-01_r8,  4.948000e-01_r8,  3.837000e-01_r8, &
  2.759000e-01_r8,  1.821000e-01_r8,  1.095000e-01_r8, &
  5.919999e-02_r8,  2.800000e-02_r8,  1.130003e-02_r8, &
  3.899990e-03_r8/)
